# Working with Files

High-throughput computing often involves analyzing data stored in files.
HTMap provides a way of specifying which files to send where, but we have to know a little bit about how HTCondor thinks about file transfer for this to work.

## File Transfer

The critical thing to keep in mind is this: **any files or directories that you transfer to the execute node are placed in the a directory that will become the working directory of your mapped function**.
That means that when you're telling HTMap what to transfer, you need to use **local paths**, but when you're writing your function itself, **everything will be relative to the current working directory**.

To see this, let's make a simple function that returns the name and contents of the current working directory:

In [1]:
import htmap

from pathlib import Path  # pathlib rules, os.path drools

# mapped functions must take at least one argument
# if you don't need one, _ is a good signal for "I don't need this"
def where_am_i(_): 
    cwd = Path.cwd()
    contents = list(cwd.iterdir())
    
    return cwd, contents

In [2]:
map = htmap.map(where_am_i, [None])
# same as above: None is a good argument when you don't have anything to send in

# extract the output of the first-and-only component
# and unpack it's contents using tuple unpacking
cwd, contents = map.get(0) 

print('current working directory:', cwd)
print('contents:')
for c in contents:
    print(c)

current working directory: /home/jovyan/.condor/jupyter_htcondor_state/execute/dir_5772
contents:
/home/jovyan/.condor/jupyter_htcondor_state/execute/dir_5772/_condor_stderr
/home/jovyan/.condor/jupyter_htcondor_state/execute/dir_5772/.chirp.config
/home/jovyan/.condor/jupyter_htcondor_state/execute/dir_5772/condor_exec.exe
/home/jovyan/.condor/jupyter_htcondor_state/execute/dir_5772/.job.ad
/home/jovyan/.condor/jupyter_htcondor_state/execute/dir_5772/_condor_stdout
/home/jovyan/.condor/jupyter_htcondor_state/execute/dir_5772/.machine.ad
/home/jovyan/.condor/jupyter_htcondor_state/execute/dir_5772/_htmap_transfer
/home/jovyan/.condor/jupyter_htcondor_state/execute/dir_5772/0.in
/home/jovyan/.condor/jupyter_htcondor_state/execute/dir_5772/func


As you can see, the working directory during execution isn't our local working directory.

Now let's transfer a file in there and see where it ends up.
Make a local file in a subdirectory relative to your local working directory.

In [3]:
subdir = Path.cwd() / 'htmap_file_tutorial'
print(subdir)
file = subdir / 'hello-world.txt'
print(file)

/home/jovyan/work/htmap_file_tutorial
/home/jovyan/work/htmap_file_tutorial/hello-world.txt


In [4]:
subdir.mkdir(exist_ok = True)
file.write_text('Hello world!')

12

To tell HTMap to transfer this file, we need to pass an instance of [htmap.MapOptions](../api.rst#htmap.MapOptions) to our map call.

In [5]:
map = htmap.map(
    where_am_i,
    [None],
    map_options = htmap.MapOptions(
        input_files = [file],  # the entire local path to the file!
    ),
)

cwd, contents = map.get(0)
print('current working directory:', cwd)
print('contents:')
for c in contents:
    print(c)

current working directory: /home/jovyan/.condor/jupyter_htcondor_state/execute/dir_5789
contents:
/home/jovyan/.condor/jupyter_htcondor_state/execute/dir_5789/_condor_stderr
/home/jovyan/.condor/jupyter_htcondor_state/execute/dir_5789/.chirp.config
/home/jovyan/.condor/jupyter_htcondor_state/execute/dir_5789/condor_exec.exe
/home/jovyan/.condor/jupyter_htcondor_state/execute/dir_5789/.job.ad
/home/jovyan/.condor/jupyter_htcondor_state/execute/dir_5789/_condor_stdout
/home/jovyan/.condor/jupyter_htcondor_state/execute/dir_5789/.machine.ad
/home/jovyan/.condor/jupyter_htcondor_state/execute/dir_5789/hello-world.txt
/home/jovyan/.condor/jupyter_htcondor_state/execute/dir_5789/_htmap_transfer
/home/jovyan/.condor/jupyter_htcondor_state/execute/dir_5789/0.in
/home/jovyan/.condor/jupyter_htcondor_state/execute/dir_5789/func


In the `input_files` keyword argument we passed the **local** path, stored in the variable `file`.
But on the execute node, the file shows up in the working directory.

For example, to read the file, we would need to do this:

In [6]:
def hello_world(_):
    return Path('hello-world.txt').read_text()
    # relative path on the execute node, 
    # because the file in the current working directory there
    

In [7]:
map = htmap.map(
    hello_world,
    [None],
    map_options = htmap.MapOptions(
        input_files = [file],  # the absolute local path
    ),
)
print(map.get(0))

Hello world!


To summarize:

* In [htmap.MapOptions](../api.rst#htmap.MapOptions), paths are **local paths on the submit node**, but they're all dumped into the **current working directory on the execute node**.
* In map inputs and map function bodies, paths are **local paths on the execute node**.

Beyond that, it's simple.
Anything you can do with a file locally, you can do on the execute node: with one caveat.
HTMap does not support custom output files.
Therefore, if you want to bring back output data, you'll need to actually return it from your mapped function (or dump the data somewhere else over the internet).

## Fixed and Variadic Inputs Files

HTMap supports two kinds of input files: 

* **Fixed input files**, which are sent to every component of the map
* **Variadic input files**, which are mapped over just like your function inputs are.

To show the different between them, let's write a functions that checks whether the contents of a "test" file are the same as the contents of a "master" file.
We'll make a map where we pass a few files in as test files (these will be **variadic**), with a single master file to compare them against (**fixed**).

In [8]:
def compare_files(test_file, master_file = None):
    test = Path(test_file)
    master = Path(master_file)
    
    return test.read_text() == master.read_text()

And here's some input files:

In [9]:
# use local relative paths to keep ourselves sane

subdir = Path.cwd() / 'htmap_file_tutorial'
master_file = subdir / 'master.txt'
test_files = [subdir / '{}.txt'.format(letter) for letter in 'abcdef']
print('master file:', master_file)
print('test files:')
for file in test_files:
    print(file)

master file: /home/jovyan/work/htmap_file_tutorial/master.txt
test files:
/home/jovyan/work/htmap_file_tutorial/a.txt
/home/jovyan/work/htmap_file_tutorial/b.txt
/home/jovyan/work/htmap_file_tutorial/c.txt
/home/jovyan/work/htmap_file_tutorial/d.txt
/home/jovyan/work/htmap_file_tutorial/e.txt
/home/jovyan/work/htmap_file_tutorial/f.txt


Now let's write something to those files.
We'll put `'hello'` in the master file, and alternate between `'hello'` and `'goodbye'` in the test files.

In [10]:
subdir.mkdir(exist_ok = True)

master_file.write_text('hello')

for index, test_file in enumerate(test_files):
    test_file.write_text('hello' if index % 2 == 0 else 'goodbye')

One last piece of setup: the function inputs can't be the actual path objects, because `compare_files` is expecting the name of the input files to come in as plain strings.
We can get that from a `Path` object like this:

In [11]:
master_file.name

'master.txt'

This will be by far our most complicated map yet.
I'll add some comments to each line to try to clarify what everything does.

In [12]:
map = htmap.map(
    compare_files,
    (file.name for file in test_files),  # these are the inputs we map over: they become the first argument to compare_files
    master_file = master_file.name,  # this keyword argument is passed to all components
    map_options = htmap.MapOptions(
        fixed_input_files = [master_file],  # here we give it the actual Path object, representing the local path
        input_files = test_files,  # the list of test file Path objects, which will be mapped over
    ),
)

Note that we had to manually "align" the function arguments and the `input_files`.
This is an unfortunate duplication of effort and information.
If you ever figure out a way to fix that, please submit a pull request...

In [13]:
print(list(map))

[True, False, True, False, True, False]


Huzzah!

If you're following along at home, run this block to clean up the test files:

In [14]:
import shutil
shutil.rmtree(str(subdir))

The [next tutorial](map-options.ipynb) will discuss the other major use for [htmap.MapOptions](../api.rst#htmap.MapOptions): passing metadata about your map's requirements to HTCondor.